In [25]:
import pandas as pd
import requests
import re
import time
from tqdm import tqdm
import html
import json

In [3]:
path = "/Users/alessandromolinarroet/Desktop/programming_database_web_applications_2/database/data/tum_course_data/tum_courses_step1_collection.csv"
df = pd.read_csv(path)

In [14]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json'
}
descriptions = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Scraping TUM Descriptions"):
    course_url = str(row['URL'])
    
    try:
        course_id_match = re.search(r'courses/(\d+)', course_url)
        if not course_id_match:
            descriptions.append("ID not found in URL")
            continue
        course_id = course_id_match.group(1)
        api_url = f"https://campus.tum.de/tumonline/ee/rest/slc.tm.cp/student/courses/{course_id}"
        
        # Fetch data from API
        response = requests.get(api_url, headers=headers, timeout=10)
        if response.status_code != 200:
            descriptions.append(f"Error: Status {response.status_code}")
            continue
        data = response.json()
        
        resources = data.get('resource', [])
        if not resources:
            descriptions.append("No data available")
            continue
        
        description_data = (
            resources[0]
            .get('content', {})
            .get('cpCourseDetailDto', {})
            .get('cpCourseDescriptionDto', {})
        )
        
        content_field = description_data.get('courseContent', {})
        content = content_field.get('value', '')  # Default value
        
        content_translations = content_field.get('translations', {}).get('translation', [])
        for trans in content_translations:
            if trans.get('lang') == 'en' and trans.get('value'):
                content = trans.get('value')
                break
        
        objectives_field = description_data.get('courseObjective', {})
        objectives = objectives_field.get('value', '')
        
        objectives_translations = objectives_field.get('translations', {}).get('translation', [])
        for trans in objectives_translations:
            if trans.get('lang') == 'en' and trans.get('value'):
                objectives = trans.get('value')
                break

        parts = []
        if content and len(content) > 1:
            parts.append(f"CONTENT:\n{content}")
        if objectives and len(objectives) > 1:
            parts.append(f"OBJECTIVES:\n{objectives}")
        
        if parts != []:
            final_description = "\n\n".join(parts) 
        else: 
            final_description = "No information available"
        descriptions.append(final_description)

        
    except Exception as e:
        print(f"Error processing {course_url}: {e}")
        descriptions.append(f"Error: {str(e)}")


df['Description'] = descriptions
df.to_csv("courses.csv", index=False)


Scraping TUM Descriptions: 100%|██████████| 3768/3768 [11:23<00:00,  5.51it/s]


In [24]:
(df["Description"] == "No information available").value_counts()
df.head()

,Title,Semester,Description,Skills,URL
0,Dodo Alive! - Resurrecting the Dodo with Robot...,2025 W,CONTENT:\nImagine that you are at the natural ...,Pending...,https://campus.tum.de/tumonline/ee/ui/ca2/app/...
1,Biochemie reaktiver Sauerstoffspezies und frei...,2025 W,CONTENT:\nThe main focus of the course is on b...,Pending...,https://campus.tum.de/tumonline/ee/ui/ca2/app/...
2,Instationäre hygrothermische Berechnungsverfahren,2025 W,No information available,Pending...,https://campus.tum.de/tumonline/ee/ui/ca2/app/...
3,A Different Kind of Game Jam! Reflecting Scien...,2025 W,CONTENT:\nThis course offers an interdisciplin...,Pending...,https://campus.tum.de/tumonline/ee/ui/ca2/app/...
4,"A Google-Earth Perspective on Nutrition, Healt...",2025 W,No information available,Pending...,https://campus.tum.de/tumonline/ee/ui/ca2/app/...


In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("GEMINI_API_KEY environment variable not set")

In [64]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash-lite")

BATCH_SIZE = 50 
DAILY_LIMIT = 100 

with open("../extraction/lists/skills.txt", 'r', encoding='utf-8') as f:
    skill_list = [line.strip() for line in f if line.strip()]

unprocessed_indices = df[df['Skills'] == 'Pending...'].index.tolist()
for i in tqdm(range(0, len(unprocessed_indices), BATCH_SIZE)):

    current_request_count = i // BATCH_SIZE
    if current_request_count >= DAILY_LIMIT:
        print(f"\nDaily quota of {DAILY_LIMIT} requests reached. Save and resume tomorrow!")
        break

    batch_indices = unprocessed_indices[i : i + BATCH_SIZE]
    batch_rows = df.loc[batch_indices]

    courses_text = ""
    for idx, row in batch_rows.iterrows():
        desc = str(row['Description'])[:350].replace("\n", " ")
        courses_text += f"ID: {idx} | Title: {row['Title']} | Desc: {desc}\n---\n"

    prompt = f"""
    Identify the most relevant skills for these {len(batch_rows)} courses from the provided list.
    MAX 10 skills per course, MIN 1 skill per course. 
    You should select only skill that are probably teached in the courses.

    Skills Bank: {", ".join(skill_list)}

    Courses to Analyze:
    {courses_text}

    Return a JSON object where:
    - Keys are the Course IDs (strings)
    - Values are JSON arrays of strings (the skills)
    Example: {{"{batch_indices[0]}": ["Python", "SQL"], ...}}
    """

    try:
        response = model.generate_content(
            prompt, 
            generation_config={"response_mime_type": "application/json"}
        )
        batch_results = json.loads(response.text)
        
        for course_id_str, skills in batch_results.items():
            course_id = int(course_id_str)
            if course_id in df.index:
                df.at[course_id, "Skills"] = skills if isinstance(skills, list) else []
                
    except Exception as e:
        print(f"\nBatch starting at index {batch_indices[0]} failed: {e}")
        time.sleep(10)

    time.sleep(5) 

    # Saves 
    if (current_request_count + 1) % 2 == 0:
        df.to_csv("courses_progress.csv", index=False)


df.to_csv("courses_progress.csv", index=False)
print("Processing complete for today.")

  0%|          | 0/72 [00:00<?, ?it/s]

 74%|███████▎  | 53/72 [07:50<02:51,  9.00s/it]


Batch starting at index 2842 failed: Expecting property name enclosed in double quotes: line 52 column 1 (char 4070)


100%|██████████| 72/72 [10:40<00:00,  8.89s/it]

Processing complete for today.


In [84]:
from dotenv import load_dotenv
import os, json, time
import google.generativeai as genai
from tqdm import tqdm

# --- Gemini setup ---
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash-lite")

BATCH_SIZE = 10

# --- Load skills bank ---
with open("../extraction/lists/skills.txt", "r", encoding="utf-8") as f:
    skill_list = [s.strip() for s in f if s.strip()]

# --- Find rows to retry ---
todo_idx = df[
    (df["Skills"] == "Pending...") |
    (df["Skills"].astype(str) == "[]")
].index.tolist()

print(f"Remaining courses to process: {len(todo_idx)}")

# --- Process ---
for i in tqdm(range(0, len(todo_idx), BATCH_SIZE)):
    batch_idx = todo_idx[i:i + BATCH_SIZE]
    batch = df.loc[batch_idx]

    courses_text = ""
    for idx, row in batch.iterrows():
        desc = str(row["Description"])[:350].replace("\n", " ")
        courses_text += f"ID: {idx} | {row['Title']} | {desc}\n---\n"

    prompt = f"""
Identify relevant skills (1–10) per course.

Rules:
- Use ONLY skills from the bank
- Use the EXACT numeric ID shown
- Return JSON ONLY

Skills Bank:
{", ".join(skill_list)}

Courses:
{courses_text}

Correct format:
{{"123": ["Python", "SQL"]}}
"""

    try:
        response = model.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"}
        )
        results = json.loads(response.text)

        # --- CASE 1: Correct dict output ---
        if isinstance(results, dict):
            for k, v in results.items():
                if str(k).isdigit():
                    df.at[int(k), "Skills"] = v if isinstance(v, list) else []

        # --- CASE 2: Model returned a list (fallback) ---
        elif isinstance(results, list):
            for course_id, skills in zip(batch_idx, results):
                df.at[course_id, "Skills"] = skills if isinstance(skills, list) else []

        else:
            print(f"Unexpected response format at batch {batch_idx[0]}")

    except Exception as e:
        print(f"Failed batch starting at {batch_idx[0]}:", e)

    time.sleep(5)

# --- Save ---
df.to_csv("courses_progress.csv", index=False)
print("Processing complete.")


Remaining courses to process: 292


 93%|█████████▎| 28/30 [02:48<00:11,  5.78s/it]

Failed batch starting at 3751: Invalid control character at: line 1 column 155 (char 154)


100%|██████████| 30/30 [03:00<00:00,  6.00s/it]

Processing complete.


In [97]:
import ast

skills_counts = (
    df["Skills"].apply(ast.literal_eval)
    .explode()
    .value_counts()
)

print(skills_counts)

Skills
Analytics                         257
MATLAB                            169
Python                            132
Business software applications     95
ANSYS                              89
                                 ... 
Product documentation               1
ISO certification                   1
Additives                           1
Monte Carlo simulation              1
Demand modeling                     1
Name: count, Length: 5447, dtype: int64
